# Free Energies from Voronoi Tessellation 
##### For post-processing data from Voronoi Tessellation Constrainted SMCV/FTSM simulations.

### Zilin Song, 2021 June 17,
##### Ph.D. student, SMU Chemistry (Theoretical)

---
## I. Basic HOWTOs: Compute free energy with Voronoi Tessellation.


This part of codes are used for post-processing the crossing attempts from  
**Voronoi Tessllation _constrainted_ String Method** simulations,  
and to derive the Free Energies of each Voronoi cell.

Note that this part of code only deals with the data from the ***'Hard Wall (HW)'*** formulism,  
In HW formulism, the MD replicas are immediately returned to the original cell when it gets into the neightboring cell (crossed the Voronoi boundaries).  
Technically this is done by inverting its velocity once the crossing is detected. 

The underlying theory is described in **Section IV.A** of  

**_E. Vanden-Eijnden and M. Venturoli, J. Chem. Phys. 130, 19 (2009):  
"Revisiting the finite temperature string method for the calculation of reaction tubes and free energies"_**  

The practical algorithm is briefly described below.

**Denote**
> $R$ as the total number of Voronoi cells on the string;  
> $n_i$ as the total number of simulation steps of Voronoi cell $i$;  
> $\Delta t$ as the timestep of MD integration during the simultion, this is assume to be equal in all cells;  
>
> $N^{n}_{i\rightarrow{j}}$ as the number of crossing attempt from cell $i$ to cell $j$, during the $n_i$ simulation steps;  
> $\pi_i$  as probability of finding the $i$-th Voronoi cell at thermal equilibrium, $\pi_i=e^{-\frac{G_i}{k_B T}}$ by definition.  

**At cell $i$, the instantaneous rate $k_{i}$ for the MD replica to escape to cell $j$ can be computed as**
> $$ \label{eq:rate_k} \tag{1}
k_{i\rightarrow{j}}=\frac{N^{n_i}_{i\rightarrow{j}}}{n_i\Delta t} 
$$

**From the Detailed Balance Condition, we have:**
> $$ \label{eq:acrossing_attempts} \tag{2}
\sum_{j, j\ne i}^{R}\pi_{j}k_{j\rightarrow{i}} = \sum_{j, j\ne i}^{R}\pi_{i}k_{i\rightarrow{j}}
$$
>> This equation states that the number of acrossing attempts on both side of any Voronoi boundary should be equal upon thermal equilibrium.  
>> Alternatively, the chances that the system enters and exits the Voronoi Cell should be equal.

**Naturally:**
> $$ \label{eq:normalization} \tag{3}
\sum_i \pi_i = 1
$$

**We may write write equation $\eqref{eq:acrossing_attempts}$ in matrix notation:**
>\begin{equation*} \tag{4-0}
\begin{vmatrix}
\pi_1               & \pi_2               & \pi_3               & \dots               & \pi_R               
\end{vmatrix} \times \begin{vmatrix}
0                   & k_{1\rightarrow{2}} & k_{1\rightarrow{3}} & \dots               & k_{1\rightarrow{R}} \\
k_{2\rightarrow{1}} & 0                   & k_{2\rightarrow{3}} & \dots               & k_{2\rightarrow{R}} \\
k_{3\rightarrow{1}} & k_{3\rightarrow{2}} & 0                   & \dots               & k_{3\rightarrow{R}} \\
\vdots              & \vdots              & \vdots              & \ddots              & \vdots              \\
k_{R\rightarrow{1}} & k_{R\rightarrow{2}} & k_{R\rightarrow{3}} & \dots               & 0                   \\
\end{vmatrix} = \begin{vmatrix}
\pi_1               & \pi_2               & \pi_3               & \dots               & \pi_R
\end{vmatrix} \times \begin{vmatrix}
\displaystyle\sum_{j, j\ne 1}^{R}k_{1\rightarrow{j}}                   & 0                   & 0                   & \dots               & 0                   \\
0                   & \displaystyle\sum_{j, j\ne 2}^{R}k_{2\rightarrow{j}}                   & 0                   & \dots               & 0                   \\
0                   & 0                   & \displaystyle\sum_{j, j\ne 3}^{R}k_{3\rightarrow{j}}                   & \dots               & 0                   \\
\vdots              & \vdots              & \vdots              & \ddots              & \vdots              \\
0                   & 0                   & 0                   & \dots               & \displaystyle\sum_{j, j\ne R}^{R}k_{R\rightarrow{j}}                   \\
\end{vmatrix} 
\end{equation*}

**Simplify accordingly:**
> \begin{equation*} \tag{4-1}
P \times K = P \times S
\end{equation*}
>\begin{equation*} \tag{4-2}
P \times (K-S) = 0
\end{equation*}
>\begin{equation*} \tag{4-3}
(K-S)^T \times P^T = 0
\end{equation*}

**Enforce equation $\eqref{eq:normalization}$ as a constraint,  
This is done by appending 1's to the last row, we have:**
>\begin{equation*} \tag{5-0}
\begin{vmatrix}
-\displaystyle\sum_{j, j\ne 1}^{R}k_{1\rightarrow{j}} & k_{2\rightarrow{1}} & k_{3\rightarrow{1}} & \dots               & k_{R\rightarrow{1}} \\
k_{1\rightarrow{2}} & -\displaystyle\sum_{j, j\ne 2}^{R}k_{2\rightarrow{j}} & k_{3\rightarrow{2}} & \dots               & k_{R\rightarrow{2}} \\
k_{1\rightarrow{3}} & k_{2\rightarrow{3}} & -\displaystyle\sum_{j, j\ne 3}^{R}k_{3\rightarrow{j}} & \dots               & k_{R\rightarrow{3}} \\
\vdots              & \vdots              & \vdots              & \ddots              & \vdots              \\
k_{1\rightarrow{R}} & k_{2\rightarrow{R}} & k_{3\rightarrow{R}} & \dots               & -\displaystyle\sum_{j, j\ne R}^{R}k_{R\rightarrow{j}} \\
1                   & 1                   & 1                   & \dots               & 1
\end{vmatrix} \times \begin{vmatrix}
\pi_1              \\ \pi_2              \\ \pi_3              \\ \vdots             \\ \pi_R               
\end{vmatrix} = \begin{vmatrix}
0                  \\ 0                  \\ 0                  \\ \vdots             \\ 0                   \\ 1
\end{vmatrix} 
\end{equation*}

**Or equivalently:**
> \begin{equation*} \tag{5-1}
A \times x = b
\end{equation*}

**Apply QR-Decomposition on the left side, we arrive at:**
> \begin{equation*} \tag{6}
Q \times R \times x = b
\end{equation*}
>> **QR-decomposition** can be achieved by most linear algebra packages (e.g. numpy, scipy, matlab).  
>> Note that for **QR-decomposition** on A yields: an **orthogonal Q** and a **upper-right triangular R**.  
>> Therefore for Q:
>> $$ Q^T Q = 1 $$

**Finally:**
> \begin{equation*} \label{eq:final} \tag{6} 
R \times \pi = Q^T \times b
\end{equation*}
>>Equation $\eqref{eq:final}$ has the form: $Ax=b$,  
which can be solved by most linear algebra packages (e.g. numpy, scipy, matlab).

**The free energy at Voronoi Cell $i$ is then:**
> \begin{equation*} \tag{7}
G_i=-k_BT\log(\pi_i)
\end{equation*}

---
## II. Codes: Compute free energy with Voronoi Tessellation

### a. Import numpy for matrix operations

In [1]:
import numpy as np

### b. Read the Voronoi Collision matrix and compute Free energy.
You should have **voro.dat** from the **HW constrainted** SM simulations.  
**voro.dat** has the following format ( _%_ is the separation mark in voro.dat):

~~~
Part one, a matrix: 
        the r-th row shows the no. crossing attempt to the c-th column / the c-th Voronoi cell.

%
Part two, a matrix: 
        the r-th row shows the crossing attempt "accepted" to the c-th column / the c-th Voronoi cell.
        In SW formulism, it is used to count the total time that a replica spent in one cell.
        This part is ignored for the HW formulism.

%
Part three, one row of numbers: 
        the total number of simulation steps at the c-th column / the c-th Voronoi cell.
~~~


In [2]:
def load_voro_dat(filedir, nrep, ):
    '''
    Read the Voronoi transition matrix from file. 
    
    Parameters
    ----------
    nrep: int
        The number of images along the string.
        Has to be set explicitly for a sanity check.
    filedir: str
        The directory of the voro.dat file produced from CHARMM HWcons-SMCV simulations.

    Returns
    -------
    collision_mat: numpy.ndarray
        Returns the voronoi collision matrix.
        the r-th row shows the no. crossing attempt to the c-th column / the c-th Voronoi cell.
        Should have the shape [nrep x nrep]
    nstep_vec: numpy.ndarray
        Returns the vector showing the total step count in the Voronoi cell.
        Should have the shape [nrep]
    '''
    
    # sanity check
    lines=open(filedir, 'r', ).readlines()
    if len(lines[0].split()) != nrep:
        print('Erroreous file specified. \nCheck Voronoi Transition Matrix file.')
        exit()
    
    collision_mat=[]
    
    # read the 1-st block: cross attempts.
    for line in lines[:nrep]:
        collision_mat.append([float(w) for w in line.split()])
    
    # read the 3-rd block: total steps.
    nstep_vec=[float(w) for w in lines[len(lines)-1].split()]
    
    return np.asarray(collision_mat), np.asarray(nstep_vec), 

def solve_voro_tessellation(colli_mat, nstep_vec, nrep, timestep=.0015, temperature=310., kb=.001985875, ):
    '''
    Compute the Free energies for each Voronoi Cell.
    
    Parameters
    ----------
    colli_mat: numpy.ndarray
        The collision matrix from voro.dat.
        Should be a (nrep x nrep) shape matrix.
    nstep_vec: numpy.ndarray
        The vector showing the total step count in the Voronoi Cell.
        Should be a (nrep) shape vector.
    nrep: int
        Number of Voronoi cells on the string.
        This is used only for sanity check..
    timestep:float
        The timestep of MD integration.
        Default value is 0.0015 in picoseconds.
    temperature:
        The temperature in Kelvins,
        Default value is 310 in Kelvins.
    kb:float
        The Boltzmann constant.
        Default value refers unit in in kval / (mol Kelvin)
        
    Returns
    -------
    fe_vec: numpy.ndarray
        Returns the free energies for the Voronoi cells
        Should have the shape (nrep)
    '''
    
    # Sanity check for array dimensions
    if colli_mat.shape != (nrep, nrep):
        print('\nWrong shape of {colli_mat}.\ncheck voro.dat.\n')
        print(colli_mat)
    elif nstep_vec.shape != (nrep, ):
        print('\nWrong shape of {nstep_vec}.\nCheck voro.dat.\n')
        print(nstep_vec)
    else:
        print('Passed sanity check, proceed...\n')
    
    # Equation (1), compute K
    k_mat=colli_mat / (nstep_vec * timestep)
    
    # Equation (4-2), compute K-S
    K_S_mat = k_mat - np.identity(nrep)*np.sum(k_mat, axis=1)
    
    # Equation (5-0),  
    #       A: (K-S)^T with equation (3) as constraints: Equivalent to the first matrix on left-side of Equation (5-1)
    #       y:     0^T with equation (3) as constraints: Equivalent to the right-side of equation (5-1).
    A_mat=np.append(K_S_mat.T, [[1 for i in range(len(nstep_vec))]], axis=0)
    y_vec=np.append(np.zeros((len(nstep_vec))), [1], axis=0)
    
    # QR-decomposition of A.
    Q, R = np.linalg.qr(A_mat)
    
    # Equation (6), Q^T * y on the right-side, 
    QT_y=np.matmul(Q.T, y_vec)
    
    # Solve Equation(6) for pi s, 
    pi_vec=np.linalg.solve(R, QT_y)
    
    # Equation (7), compute FE
    fe=-(kb*temperature)*(np.log(np.abs(pi_vec))-np.log(np.abs(pi_vec))[0])
    
    return fe

### c. Do your calculations on the scratch block below.
See III for an example with ****_voro0.dat_****.

---
## III. Here we put a testing case. 

You should have very close results to the reference free energies.

Reference data was stole from the CHARMM test cases.  
The original author of this data is Prof. Victor Ovchinnikov (Harvard Karplus Group).  
The collision matrix along a path of 8 string images comes from simulating the rotation of $\psi$ and $\phi$ of an alaine dipeptide.  

Note that the reference free energies are computed with a slightly different $k_B$ value.  
The algorithms to obtain the FE data also seems little bit different _(Sadly I m not sure cause I don't read Matlab)_.

The voro0.dat file contains:
>~~~
         0       188         0         0         0         0         0         0
       784         0        57         0         0         0         0         0
         0      1522         0        68         0         0         0         0
         0         0       659         0         3         0         0         0
         0         0         0      1029         0       408         0         3
         0         0         0         0         9         0      3724         0
         0         0         0         0         0        16         0      1400
         0         0         0         0         0         0       264         0
%
         0         0         0         0         0         0         0         0
         0         0         0         0         0         0         0         0
         0         0         0         0         0         0         0         0
         0         0         0         0         0         0         0         0
         0         0         0         0         0         0         0         0
         0         0         0         0         0         0         0         0
         0         0         0         0         0         0         0         0
         0         0         0         0         0         0         0         0
%
     49953     50000     49940     49207     49704     49997     49999     49999
~~~

The reference free energies are
> ~~~
-0.0000000000000000e+00
 8.5129298550034127e-01
 2.8095083186845056e+00
 4.1635102152130017e+00
 7.6437158367251010e+00
 5.3699464999767823e+00
 2.1209101304658047e+00
 1.1263537764471330e+00
~~~

In [3]:
def test():
    '''
    No doc for a test case.
    '''
    col_mat, nstep_vec=load_voro_dat('voro0.dat', 8, )
    fe=solve_voro_tessellation(col_mat, nstep_vec, 8, timestep=0.001, temperature=300, )
    print('The computed free energies are:')
    for fe in fe.tolist():
        print(fe)
test()

Passed sanity check, proceed...

The computed free energies are:
-0.0
0.8512895042289647
2.807492682004632
4.151788722842182
7.635676737295948
5.3625504267781565
2.1156789634253728
1.1217811249036878
